In [2]:
import numpy as np
import cv2
                    
def applyHomography(img, H):
    dIm = np.array([[0,0,img.shape[0],img.shape[0]],[0, img.shape[1], 0, img.shape[1]], [1, 1, 1, 1]])
    dImNew = np.matmul(H, dIm)
    dImNew[0] = dImNew[0]/dImNew[2]
    dImNew[1] = dImNew[1]/dImNew[2]
    
    minH = int(np.amin(dImNew[0]))
    maxH = int(np.amax(dImNew[0]))
    minW = int(np.amin(dImNew[1]))
    maxW = int(np.amax(dImNew[1]))
    
    imNew = np.zeros((maxH - minH, maxW - minW,3))
    invH = np.linalg.inv(H)
   
    for row in range(imNew.shape[0]):
        for col in range(imNew.shape[1]):
            coord = np.matmul(invH,[[row + minH], [col + minW], [1]])
            coord /= coord[2]
            if coord[0]<img.shape[0] and coord[0]>=0 and coord[1]<img.shape[1] and coord[1]>=0:
                imNew[row][col] = img[int(coord[0])][int(coord[1])] 
                
    return imNew

def getHomography(im1Pts, im2Pts):
    px = im1Pts[0, 1]
    py = im1Pts[0, 0]
    qx = im1Pts[1, 1]
    qy = im1Pts[1, 0]
    rx = im1Pts[2, 1]
    ry = im1Pts[2, 0]
    sx = im1Pts[3, 1]
    sy = im1Pts[3, 0]
    
    pxp = im2Pts[0, 1]
    pyp = im2Pts[0, 0]
    qxp = im2Pts[1, 1]
    qyp = im2Pts[1, 0]
    rxp = im2Pts[2, 1]
    ryp = im2Pts[2, 0]
    sxp = im2Pts[3, 1]
    syp = im2Pts[3, 0]
    
    t = np.array([[pxp],[pyp],[qxp],[qyp],[rxp],[ryp],[sxp],[syp]])
    P = np.array([[px, py, 1, 0, 0, 0, -px*pxp, -py*pxp],
                  [0, 0, 0, px, py, 1, -px*pyp, -py*pyp],
                  [qx, qy, 1, 0, 0, 0, -qx*qxp, -qy*qxp],
                  [0, 0, 0, qx, qy, 1, -qx*qyp, -qy*qyp],
                  [rx, ry, 1, 0, 0, 0, -rx*rxp, -ry*rxp],
                  [0, 0, 0, rx, ry, 1, -rx*ryp, -ry*ryp],
                  [sx, sy, 1, 0, 0, 0, -sx*sxp, -sy*sxp],
                  [0, 0, 0, sx, sy, 1, -sx*syp, -sy*syp]])
    pInv = np.linalg.inv(P)
    h = pInv.dot(t)
    H = np.array([[h[0, 0], h[1, 0], h[2, 0]], [h[3, 0], h[4, 0], h[5, 0]], [h[6, 0], h[7, 0], 1]])
    return H

def task1():
    im1 = cv2.imread("hw3_Task1_Images/Images/Img1.JPG")
    im2 = cv2.imread("hw3_Task1_Images/Images/Img2.jpeg")
    im3 = cv2.imread("hw3_Task1_Images/Images/Img3.JPG")

    dest1Corners = np.array([[0,0], [75, 0], [0,85], [75,85]])
    im1Corners = np.array([[644, 500], [663, 503], [643, 531], [663, 535]])

    dest2Corners = np.array([[0,0], [84, 0], [0,74], [84,74]])
    im2Corners = np.array([[383, 577], [593, 552], [381, 834], [607, 922]])

    dest3Corners = np.array([[0,0], [55, 0], [0,36], [55,36]])
    #im3Corners = np.array([[689, 745], [1771, 784], [739, 2099], [1774, 1617]])
    im3Corners = np.array([[2063, 700], [2665, 718], [2097, 1479], [2695, 1334]])

    H1 = getHomography(im1Corners,dest1Corners)
    H2 = getHomography(im2Corners,dest2Corners)
    H3 = getHomography(im3Corners,dest3Corners)

    cv2.imwrite('Results/Pt2Pt1.jpg', applyHomography(im1, H1))
    cv2.imwrite('Results/Pt2Pt2.jpg', applyHomography(im2, H2))
    cv2.imwrite('Results/Pt2Pt3.jpg', applyHomography(im3, H3))

def processTask2(im1, im1Corners):    
    lineTop = np.cross(im1Corners[0], im1Corners[1])
    lineBottom = np.cross(im1Corners[2], im1Corners[3])
    pt1 = np.cross(lineTop, lineBottom)
    pt1 = pt1 / pt1[2]
    print(pt1)
    
    lineLeft = np.cross(im1Corners[0],  im1Corners[2])
    lineRight = np.cross(im1Corners[1],  im1Corners[3])
    pt2 = np.cross(lineLeft, lineRight)
    pt2 = pt2 / pt2[2]
    print(pt2)
    
    vanishingLine = np.cross(pt1, pt2)
    print(vanishingLine)
    #vanishingLine = vanishingLine / vanishingLine[2]
    H = np.array([[1, 0, 0], [0, 1, 0], vanishingLine])
    print(H)
    #H = H.transpose()
    
    remProj = applyHomography(im1, H)
    return remProj
    
    
def task2():
    im1 = cv2.imread("hw3_Task1_Images/Images/Img1.JPG")
    im1Corners = np.array([[644, 500, 1], [663, 503, 1], [643, 531, 1], [663, 535, 1]])
    task2Result = processTask2(im1, im1Corners)
    cv2.imwrite('Results/TwoStep1.jpg', task2Result)
    
    """im2 = cv2.imread("hw3_Task1_Images/Images/Img2.jpeg")
    im2Corners = np.array([[383, 577, 1], [593, 552, 1], [381, 834, 1], [607, 922, 1]])
    task2Result = processTask2(im2, im2Corners)
    cv2.imwrite('Results/TwoStep2.jpg', task2Result)
    
    im3 = cv2.imread("hw3_Task1_Images/Images/Img3.JPG")
    im3Corners = np.array([[2063, 700, 1], [2665, 718, 1], [2097, 1479, 1], [2695, 1334, 1]])
    task2Result = processTask2(im3, im3Corners)
    cv2.imwrite('Results/TwoStep3.jpg', task2Result)"""
                           
task1()
#task2()

<ipython-input-2-4993c38398ae>:21: RuntimeWarning: divide by zero encountered in true_divide
  coord /= coord[2]
<ipython-input-2-4993c38398ae>:21: RuntimeWarning: invalid value encountered in true_divide
  coord /= coord[2]
